In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
import seaborn as sns

warnings.filterwarnings('ignore')
print('TensorFlow Version ' + tf.__version__)

C:\Users\PC\anaconda3\envs\tensorflow-gpu\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\PC\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\PC\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions

TensorFlow Version 2.10.1


In [2]:
IMAGE_SIZE = 224
BATCH_SIZE = 8
EPOCHS = 5

In [ ]:
def data_augment(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k = 3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k = 2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k = 1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower = .7, upper = 1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower = .8, upper = 1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta = .1)
        
    return image

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                          samplewise_center = True,
                                                          samplewise_std_normalization = True,
                                                          validation_split = 0.2,
                                                          preprocessing_function = data_augment)

train_gen = datagen.flow_from_directory(
                                        'D:/DesktopMal/malimg_dataset/train',
                                        batch_size = BATCH_SIZE,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = True,
                                        class_mode = 'categorical',
                                        target_size = (224, 224))

valid_gen = datagen.flow_from_directory(
                                        'D:/DesktopMal/malimg_dataset/train',
                                        batch_size = BATCH_SIZE,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = True,
                                        class_mode = 'categorical',
                                        target_size = (224, 224))

test_gen = datagen.flow_from_directory(
                                        'D:/DesktopMal/malimg_dataset/train',
                                        batch_size = BATCH_SIZE,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = True,
                                        class_mode = 'categorical',
                                        target_size = (224, 224))

In [ ]:
classes = train_gen.class_indices
t_name=[k for  k,v in classes.items()]
classes = dict((v,k) for k,v in classes.items())
# classes = [classes[k] for k in classes]
classes

In [ ]:
t_name

In [5]:
from vit_keras import vit
vit_model = vit.vit_b16(
        image_size = 128,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 25)

In [ ]:
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(11, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(25, 'softmax')
    ],
    name = 'vision_transformer')

model.summary()

In [6]:
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=32, activation='selu', kernel_initializer='glorot_uniform'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dense(units=48, activation='softplus', kernel_initializer='glorot_normal'),
tf.keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.25),
tf.keras.layers.Dense(units=112, activation='relu', kernel_initializer='lecun_uniform'),

tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(25, 'softmax')
    ],
    name = 'vision_transformer')

model.summary()

Model: "vision_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b16 (Functional)        (None, 768)               85697280  
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 32)                24608     
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 48)                1584      
                                                                 
 batch_normalization_1 (Batc  (None, 48)               192       
 hNormalization)                                

In [ ]:
learning_rate = 1e-4

optimizer = tfa.optimizers.RectifiedAdam(learning_rate = learning_rate)

model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2), 
              metrics = ['accuracy'])

STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                factor = 0.2,
                                                patience = 2,
                                                verbose = 1,
                                                min_delta = 1e-4,
                                                min_lr = 1e-6,
                                                mode = 'max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                min_delta = 1e-4,
                                                patience = 5,
                                                mode = 'max',
                                                restore_best_weights = True,
                                                verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './model.hdf5',
                                                 monitor = 'val_accuracy', 
                                                 verbose = 1, 
                                                 save_best_only = True,
                                                 save_weights_only = True,
                                                mode = 'max')

callbacks = [earlystopping, reduce_lr, checkpointer]

model.fit(X_train,y_train,
         #steps_per_epoch = STEP_SIZE_TRAIN,
         #validation_data = valid_gen,
         #validation_steps = STEP_SIZE_VALID,
         epochs = EPOCHS, validation_split=0.1)
         #callbacks = callbacks)

#model.fit(x = train_gen,validation_data = valid_gen)

model.save('model.h5')

In [7]:
from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.001, momentum = 0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
historyCNN_ANN = model.fit(X_train,y_train, epochs=50, validation_data=(X_test,y_test),verbose=1,batch_size=32)

Epoch 1/50
704/704 [==============================] - 244s 254ms/step - loss: 1.7720 - accuracy: 0.4823 - val_loss: 0.4096 - val_accuracy: 0.8572
Epoch 2/50
704/704 [==============================] - 177s 251ms/step - loss: 0.4655 - accuracy: 0.8502 - val_loss: 0.2581 - val_accuracy: 0.9056
Epoch 3/50
704/704 [==============================] - 176s 250ms/step - loss: 0.2969 - accuracy: 0.9000 - val_loss: 0.2080 - val_accuracy: 0.9356
Epoch 4/50
704/704 [==============================] - 176s 250ms/step - loss: 0.2069 - accuracy: 0.9362 - val_loss: 0.1236 - val_accuracy: 0.9476
Epoch 5/50
704/704 [==============================] - 176s 250ms/step - loss: 0.1548 - accuracy: 0.9532 - val_loss: 0.0964 - val_accuracy: 0.9660
Epoch 6/50
704/704 [==============================] - 176s 250ms/step - loss: 0.1226 - accuracy: 0.9646 - val_loss: 0.0682 - val_accuracy: 0.9792
Epoch 7/50
704/704 [==============================] - 177s 251ms/step - loss: 0.0968 - accuracy: 0.9724 - val_loss: 0.0605 -

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
# CNN'in train/validation loss ve accuraccy'lerindeki değişimi 
# görmek amacıyla bir fonksiyon yazıyorum

def plot_historyACC(history, model):
    plt.subplots(figsize=(7,10))
    plt.subplot(2,1,1) 
    plt.ylim(0,1)
    plt.title('Loss')
    plt.plot(history.history['loss'], color='green', alpha = 0.7, label='Train Loss')
    plt.plot(history.history['val_loss'], color='orange', alpha = 0.7, label='Validation Loss')
    plt.legend() 
    plt.subplot(2,1,2) 
    plt.title('Accuracy')
    plt.plot(history.history['accuracy'], color='green', alpha = 0.7, label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], color='orange', alpha = 0.7, label='Validation Accuracy')
    plt.ylim(0.5,1)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_historyACC(historyCNN_ANN, model)

In [ ]:
Y_pred = model.predict(test_gen)


In [ ]:
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
y_pred[50],test_gen.classes[50]

In [ ]:
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_gen.classes, y_pred))
print('Classification Report')
print(classification_report(test_gen.classes, y_pred, target_names=t_name))